# EfficientNet B0

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

device = torch.device("cpu")
print(f"Using device: {device}")

train_dir = "dataset/Training"
test_dir = "dataset/Testing"
batch_size = 16  
image_size = 224  # EfficientNet-B0 size

# image transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

num_classes = len(train_dataset.classes)
print(f"Detected classes: {train_dataset.classes}")

weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device, memory_format=torch.contiguous_format)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
    return 100 * correct / total

def train_model(model, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(device, memory_format=torch.contiguous_format)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
        train_acc = 100 * correct / total
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, "
              f"Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")

train_model(model, train_loader, val_loader, epochs=5)

print(f"\n🧪 Final Test Accuracy: {test_acc:.2f}%")


Using device: cpu
Detected classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


Epoch 1: 100%|██████████| 304/304 [34:03<00:00,  6.72s/it]


Epoch 1: Loss=0.3877, Train Acc=88.03%, Val Acc=96.50%


Epoch 2: 100%|██████████| 304/304 [57:22<00:00, 11.32s/it] 


Epoch 2: Loss=0.1249, Train Acc=95.70%, Val Acc=97.43%


...

Epoch 3: Loss=0.0582, Train Acc=98.41%, Val Acc=98.48%


Epoch 4: 100%|██████████| 304/304 [2:46:05<00:00, 32.78s/it]    


Epoch 4: Loss=0.0412, Train Acc=98.76%, Val Acc=98.95%


Epoch 5: 100%|██████████| 304/304 [1:31:48<00:00, 18.12s/it]   


Epoch 5: Loss=0.0381, Train Acc=98.85%, Val Acc=98.37%

🧪 Final Test Accuracy: 98.63%


In [4]:
#Save the model
torch.save(model.state_dict(), "efficientnet_b0_model_5epochs_9863.pth")

# EfficientNetV2 B0

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import timm  # Hugging Face's PyTorch Image Models

print(timm.list_models("efficientnetv2*"))

device = torch.device("cpu")
print(f"Using device: {device}")

train_dir = "dataset/Training"
test_dir = "dataset/Testing"
batch_size = 16
image_size = 224  # Required for EfficientNetV2-B0

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

num_classes = len(train_dataset.classes)
print(f"Detected classes: {train_dataset.classes}")

model = timm.create_model('efficientnetv2_rw_t', pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device, memory_format=torch.contiguous_format)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
    return 100 * correct / total

def train_model(model, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(device, memory_format=torch.contiguous_format)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
        train_acc = 100 * correct / total
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, "
              f"Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")

train_model(model, train_loader, val_loader, epochs=5)

test_acc = evaluate(model, test_loader)
print(f"\n🧪 Final Test Accuracy: {test_acc:.2f}%")

['efficientnetv2_l', 'efficientnetv2_m', 'efficientnetv2_rw_m', 'efficientnetv2_rw_s', 'efficientnetv2_rw_t', 'efficientnetv2_s', 'efficientnetv2_xl']
Using device: cpu
Detected classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


model.safetensors:   0%|          | 0.00/55.1M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 304/304 [1:15:36<00:00, 14.92s/it]


Epoch 1: Loss=0.3565, Train Acc=88.26%, Val Acc=97.55%


Epoch 2: 100%|██████████| 304/304 [1:23:17<00:00, 16.44s/it]


...

Epoch 3: Loss=0.0346, Train Acc=99.24%, Val Acc=97.55%


Epoch 4: 100%|██████████| 304/304 [1:22:46<00:00, 16.34s/it]


Epoch 4: Loss=0.0230, Train Acc=99.38%, Val Acc=99.07%


Epoch 5: 100%|██████████| 304/304 [1:22:35<00:00, 16.30s/it]


Epoch 5: Loss=0.0133, Train Acc=99.59%, Val Acc=98.83%

🧪 Final Test Accuracy: 98.63%


In [10]:
# Save the model
torch.save(model.state_dict(), "efficientnetv2_rw_t_model_5epochs.pth")